In [1]:
from utils.data_cleaning import run_listen_pipeline
from utils.data_preprocess import codic_of_index, train_test_split
from utils.evaluate import evaluate_model
from utils.cf_utils import *
from utils.models import *

import pandas as pd
import polars as pl


import faiss
from tqdm import tqdm
from sklearn.pipeline import Pipeline


/opt/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data

### DataLoding

In [2]:
data = pd.read_parquet("data/multi_event.parquet", engine="fastparquet")
items_meta = pd.read_parquet("data/items_meta.parquet")

### Train/test split

Для начала разделим данные на обучающую и тестовую выборки. Деление выполняется по временному признаку, поскольку мы работаем с последовательностями действий пользователей.

Чтобы избежать data leakage — ситуации, когда признаки, рассчитанные на исторических данных, могут «заглядывать» в будущее — вводится временной зазор (gap) между обучающей и тестовой частями.

В оригинальной статье https://www.arxiv.org/pdf/2505.22238  используется соотношение 300 дней обучения, 30 минут gap, 1 день теста.

В моих экспериментах наилучшие результаты получились при конфигурации: 80 дней обучения, 30 минут gap, 20 дней теста.

Чтобы приблизиться к условиям статьи, я планирую снижать размер тестовой выборки, сохраняя возможность варьировать объём обучающей части как гиперпараметр модели. Такой подход позволяет гибко контролировать баланс между длиной обучающего периода и реалистичностью оценки качества.

In [3]:
train_df, test_df = train_test_split(data, 1, 50, gap_size = 30)

### Очиска и кодирование данных 
Применим необходимую предобработку для очистки и подготовки данных. Подробные рассуждения о шагах и мотивации предобработки приведены в блокноте multi_event_preprocessing.
Все функции вынесены в отдельный модуль, что делает код более читаемым, модульным и удобным для повторного использования.

Кодируем uid и item_id в числовые индексы, чтобы модели и алгоритмы могли с ними работать.

Строковые ID нельзя напрямую подать в матрицу взаимодействий или Embedding-слой, поэтому мы переводим их в диапазон 0..N.
Это нужно для эффективности (память, скорость), совместимости с моделями (Matrix Factorization, Embeddings) и стабильного маппинга между train и test.

In [4]:
print(len(test_df))
print(len(train_df))

train_df, test_df , item_id_map = codic_of_index(train_df, test_df)

train_df_likes = train_df[~(train_df["event_type"] == "listen")]
test_df_likes = test_df[~(test_df["event_type"] == "listen")]

train_df_listen = train_df[train_df["event_type"] == "listen"]
test_df_listen = test_df[test_df["event_type"] == "listen"]


train_df_listen, stats = run_listen_pipeline(train_df_listen,
                                            ["remove_duplicates_by_timestamps", 
                                             "remove_not_valid_listen_data", 
                                             "cut_track_len", 
                                             "remove_rer_data"])


test_df_listen, stats = run_listen_pipeline(test_df_listen,
                                            ["clean_listen_duplicates", 
                                             "remove_not_valid_listen_data"])


train_df_likes, stats = run_listen_pipeline(train_df_likes,
                                            ["convert_reaction", 
                                             "rename_events"])


test_df_likes, stats = run_listen_pipeline(test_df_likes,
                                            ["convert_reaction", 
                                             "rename_events"])


test_df = pd.concat([test_df_likes,test_df_listen ]) 
train_df = pd.concat([train_df_likes,train_df_listen ]) 

items_meta = pd.read_parquet("data/items_meta.parquet")
items_meta["item_id"] = items_meta["item_id"].map(item_id_map)
items_meta = items_meta.drop_duplicates(subset=["item_id"])
items_meta = items_meta.dropna()


df_merged = train_df.merge(items_meta[['item_id', 'artist_id', 'album_id']], 
                      on='item_id', 
                      how='left')

print(len(test_df))
print(len(train_df))

238398
10510818
216480
7186431


In [5]:
test_df_likes = None
train_df_likes = None
train_df_listen= None
test_df_listen= None

# Models

## Baseline

#### Глобальные тренды

Самый простой базовый подход — взять топ-10 самых популярных треков. При этом возможны два варианта отбора:

* по количеству лайков,
* по количеству прослушиваний.

Реализованы обе версии, однако в качестве базовой модели (baseline) выбран именно топ-10 самых залайканных треков среди всех пользователей. Лайки считаются более надёжным сигналом предпочтений, чем прослушивания. (Для чистоты эксперимента был протестирован и вариант с прослушиваниями — его точность оказалась ниже.)



Поскольку тренды и популярные песни сменяются очень быстро, временное окно, в котором мы определяем тренд, также должно быть коротким. (*PS нужно оптимизировать выбор автоматически*)

In [37]:
model = MostPop_by_likes()

last_5days = train_df[train_df['timestamp'] > (train_df['timestamp'].max() - 5 * DAY_SECONDS)]
model.fit(last_5days)

evaluate_model(model, train_df, test_df)

100%|███████████████████████████████████████████████████████████████████████████████| 4989/4989 [00:00<00:00, 8270.71it/s]

Mean Recall@10: 0.012558422238873436
Mean NDCG@10: 0.03221620193341678


In [30]:
model = MostPop_by_listen()
model.fit(last_5days)

evaluate_model(model, train_df, test_df)

100%|███████████████████████████████████████████████████████████████████████████████| 4989/4989 [00:00<00:00, 9038.65it/s]

Mean Recall@10: 0.010909877328199646
Mean NDCG@10: 0.028976641999167746


#### Перносальный топ для пользователя 

Пользователь чаще всего продолжает слушать те треки, которые были популярны для него в последнее время. Если человек регулярно возвращался к композиции в течение последних 20 дней, с высокой вероятностью он будет слушать её и далее — это отражает его актуальные предпочтения.
Такой подход позволяет формировать рекомендации, опираясь на краткосрочные интересы, и, например, делать подборки из треков одного альбома или исполнителя, которые пользователь активно слушал в недавнем прошлом.

Сейчас аналитика делается за последние 20 дней (*PS нужно оптимизировать выбор автоматически*)

In [32]:

last_20days = df_merged[df_merged['timestamp'] > (df_merged['timestamp'].max() - 20 * DAY_SECONDS)]
model = RecentAactivityBasedRecommendation()
model.fit(last_20days)


evaluate_model(model, train_df, test_df)

100%|████████████████████████████████████████████████████████████████████████████████| 4989/4989 [00:34<00:00, 146.39it/s]

Mean Recall@10: 0.07706722383689496
Mean NDCG@10: 0.17755570717695981


In [65]:
import pandas as pd


class NewItemsLastNDays:
    def __init__(self, days: int = 5, time_col: str = "timestamp"):
        """
        days     — за сколько последних дней считать новинки
        time_col — название колонки с временем (у тебя 'timestamp')
        """
        self.days = days
        self.time_col = time_col
        self.rec = []
        self.weights = []

    def fit(self, df: pd.DataFrame):
        df = df.copy()

        # находим глобальное "сейчас" по данным — максимальный timestamp в логе
        max_ts = df[self.time_col].max()

        HOUR_SECONDS = 60 * 60
        DAY_SECONDS = 24 * HOUR_SECONDS

        # порог: новинки — которые ВПЕРВЫЕ появились после этого времени
        cutoff = max_ts - DAY_SECONDS * self.days

        history = df[df["timestamp"]<cutoff]["item_id"].unique().tolist()
        current = df[df["timestamp"]>=cutoff]["item_id"].unique().tolist()

        new = list(set(current) - set(history))
        
        
        new_items = df[df["item_id"].isin(new)]


        self.rec = new_items["item_id"].value_counts().index.tolist()
        print(self.rec[:10])
        # веса — можно взять просто timestamp в секундах (чем больше, тем новее)
        self.weights = new_items["item_id"].value_counts().tolist()

    def recommend(self, uid, k: int = 10):
        """
        uid здесь не используется — глобальная витрина новинок,
        как и в MostPop_by_likes.
        """
        return self.rec[:k], self.weights[:k]


In [66]:
model = NewItemsLastNDays()
model.fit(train_df)


evaluate_model(model, train_df, test_df)

[9486, 3477, 417, 419, 169987, 273584, 105419, 14564, 14566, 39919]


100%|██████████████████████████████████████████████████████████████████████████████| 4989/4989 [00:00<00:00, 10046.39it/s]

Mean Recall@10: 0.0066837662901712
Mean NDCG@10: 0.02053104239488795


In [102]:
class FavoriteArtistsNewReleases:
    def __init__(self):
        pass

    def fit(self, df_merged):

        self.days = 30
        
        # находим глобальное "сейчас" по данным — максимальный timestamp в логе
        max_ts = df_merged["timestamp"].max()

        HOUR_SECONDS = 60 * 60
        DAY_SECONDS = 24 * HOUR_SECONDS

        # порог: новинки — которые ВПЕРВЫЕ появились после этого времени
        cutoff = max_ts - DAY_SECONDS * self.days

        history = df_merged[df_merged["timestamp"]<cutoff]["item_id"].unique().tolist()
        current = df_merged[df_merged["timestamp"]>=cutoff]["item_id"].unique().tolist()

        new = list(set(current) - set(history))
        
        self.new_items = df_merged[df_merged["item_id"].isin(new)]
        
        
        likes = df_merged[df_merged["event_type"] == "like"].copy()

        listen = (
                df_merged[df_merged["event_type"] == "listen"]
                .groupby(['uid', 'item_id'])
                .agg({
                    'timestamp': 'count',
                    'artist_id': 'first',
                    'album_id': 'first',
                    
                })
                .reset_index()
            )
        
        listen = listen[listen["timestamp"]>5]

        likes = pd.concat([likes, listen])

        self.likes = likes

        
    def ger_rec(self, uid, flag= False):
        N_ARTISTS = 100   # сколько самых популярных артистов
        N_TRACKS = 100    # сколько треков на каждого артиста

        user_likes =  self.likes[self.likes["uid"] == uid]
        user_item = list(user_likes["item_id"].unique())
    
        # 2. Считаем, сколько лайков у каждого артиста
        artist_like_counts = (
            user_likes.groupby("artist_id")["item_id"]
            .count()
            .sort_values(ascending=False)
            .reset_index(name="artist_likes")
        )
        
        # 3. Берём топ-10 артистов
        top_artists = artist_like_counts.head(N_ARTISTS).copy()
        top_artists["artist_rank"] = range(1, len(top_artists) + 1)

        favorite_new = self.new_items[self.new_items["artist_id"].isin(top_artists["artist_id"])]

        favorite_new = favorite_new.copy()
        order = top_artists["artist_id"].tolist()

        favorite_new["artist_id"] = pd.Categorical(favorite_new["artist_id"], categories=order, ordered=True)


        
        favorite_new = favorite_new.sort_values("artist_id")

        return favorite_new["item_id"].value_counts().index.tolist(), favorite_new["item_id"].value_counts().tolist()
 

    def recommend(self, uid, k = 10):
        rec, weights = self.ger_rec(uid) 
        return rec[:k], weights[:k]

In [103]:
model = FavoriteArtistsNewReleases()
model.fit(df_merged)


evaluate_model(model, train_df, test_df)

100%|████████████████████████████████████████████████████████████████████████████████| 4989/4989 [00:43<00:00, 115.01it/s]

Mean Recall@10: 0.01800076613307598
Mean NDCG@10: 0.0475082237130265


## Collaborative filtering

### ALS

AlternatingLeastSquares (ALS) — это модель матричной факторизации для неявных откликов (implicit feedback).
Она раскладывает матрицу взаимодействий пользователей и объектов на две матрицы эмбеддингов — пользователей и айтемов — так, чтобы их скалярное произведение предсказывало интерес.
Обучается поочерёдным решением задач наименьших квадратов: сначала фиксирует айтемы и обновляет пользователей, потом наоборот.


В одной итерации ALS поочерёдно пересчитывает вектора пользователей и айтемов.
Сначала, при фиксированных айтемах, решаются задачи наименьших квадратов для всех пользователей;
потом, при фиксированных пользователях — для всех айтемов.
Это повторяется несколько итераций, пока модель не сойдётся.

Для обучения ALS требуется подготовить данные в формате матрицы взаимодействий, где каждая ячейка отражает степень интереса пользователя к объекту. Поскольку в нашем случае используются implicit-сигналы, нам необходимо преобразовать исходные события (прослушивание, лайк, дизлайк) в числовые веса.

На этапе подготовки данных мы подсчитываем, сколько раз пользователь прослушал каждый трек, а также приводим реакции к бинарному виду:
в результирующем датафрейме поле like получает значение 1,
аналогично dislike — значение 1.

В отдельном модуле реализована функция объединения данных, которая формирует итоговый датафрейм с колонками:

* признак лайка (like = 1)
* признак дизлайка (dislike = 1)
* количество прослушиваний каждой пары пользователь–трек

In [7]:
train_df_als = merge_data_by_count(train_df)

# ToDo формулу описать!!! зачем логорифирование и описать смысл коэффициентов  
train_df_als["conf"] = (train_df_als["have_like"] * 20.0) + (train_df_als["have_dislike"] * -10) + np.log1p(train_df_als["listen_count"]*(train_df_als["played_ratio_pct"]/100))


Для ALS необходима разреженная матрица вида:

(user_id, item_id) → weight


где weight — это итоговая сила взаимодействия (прослушивания + реакции).
В модуле реализована функция, которая на основе сформированного датафрейма строит такую матрицу через coo_matrix и возвращает её в подходящем для модели формате.

In [8]:
matrix = create_user_item_matrix(train_df_als)
matrix

<Compressed Sparse Row sparse matrix of dtype 'int32'
	with 3569889 stored elements and shape (9968, 468711)>

In [36]:
model = ALS()
model.fit(matrix)

100%|█████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:43<00:00,  1.43s/it]


In [37]:
evaluate_model(model, train_df, test_df)

100%|█████████████████████████████████████████████████████████████████████████████████| 4989/4989 [05:09<00:00, 16.10it/s]

Mean Recall@10: 0.0372679892386115
Mean NDCG@10: 0.095950877140579


Метод ALS работает только с пользователями и объектами, которые присутствовали в обучающей выборке. Если в тесте встречается пользователь, которого модель ранее не видела, для него невозможно построить профиль в факторизационном пространстве.

В такой ситуации ALS не способен выдать рекомендации — в классе ALS такая логика приводит к возврату пустого списка.

Поэтому при использовании ALS важно учитывать, что:

* рекомендации гарантированно работают только для пользователей из train,
* все новые или ранее невстречавшиеся пользователи будут давать «пустой» результат.

Это естественное ограничение моделей матричной факторизации, основанных на implicit-feedback данных.

### ItemItemRec

In [ ]:
model = ItemItemRec()
model.fit(matrix)

evaluate_model(model, train_df, test_df)

100%|██████████████████████████████████████████████████████████████████████████| 468711/468711 [00:38<00:00, 12231.84it/s]


У авторов в статье точность была намного выше - возможно из-за того, что обучались на всем наборе данных, а не на одном месяце

### BPR

In [9]:
# BPR ожидает ITEM×USER матрицу, поэтому транспонируем
# (это стандарт для implicit: матрица item-user)

item_user_matrix = matrix.T.tocsr()

model = BPR()
model.fit(matrix)

evaluate_model(model, train_df, test_df)

100%|█████████████████████████████████████████████████████████████████████████████████| 4989/4989 [03:53<00:00, 21.35it/s]

Mean Recall@10: 0.019885308813405535
Mean NDCG@10: 0.054025846595850816


## Content_based 

Content-Based Filtering — это метод рекомендаций, который подбирает объекты, похожие на те, что пользователь уже оценил, используя их содержательные признаки (жанры, embedding-и, особенности контента). Его сильная сторона в том, что он не зависит от других пользователей, хорошо работает для новых объектов (нет cold-start для items) и даёт легко объяснимые рекомендации. Однако у подхода есть ограничения: качество сильно зависит от самих признаков, модель не умеет учитывать тренды или контекст поведения, а рекомендации могут быть слишком однообразными и «сужать» вкусы пользователя.


#### embeddongs 

In [5]:
train_df_als = merge_data_by_count(train_df)
content_based_train = train_df_als[(train_df_als["have_dislike"]==0)][(train_df_als["listen_count"]>5) | (train_df_als["have_like"]==1)]


/tmp/ipykernel_24717/1594391976.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  content_based_train = train_df_als[(train_df_als["have_dislike"]==0)][(train_df_als["listen_count"]>5) | (train_df_als["have_like"]==1)]


Методы точного поиска ближайших соседей (kNN) требуют вычислять расстояние между вектором-запросом и каждым объектом в базе, поэтому их сложность линейная 
𝑂
(
𝑁
)
O(N), и на больших наборах данных с десятками или сотнями тысяч элементов они становятся крайне медленными: нужно выполнить миллионы скалярных произведений при каждом запросе. По этой причине в рекомендациях и поиске похожих объектов чаще используют ANN (Approximate Nearest Neighbors) — алгоритмы, которые строят специальный индекс (HNSW, Annoy, Faiss и др.) и ищут ближайших соседей не полностью перебирая весь набор, а умно обходя лишь небольшую часть точек. В результате скорость поиска ускоряется в десятки–сотни раз при минимальной потере качества, что делает ANN практически стандартом для больших эмбеддингов.

In [6]:
index, all_item_ids = create_index("filtered_embeddings.parquet", item_id_map)

added up to offset=100000, total indexed=51853
added up to offset=150000, total indexed=77962
added up to offset=200000, total indexed=103823
added up to offset=250000, total indexed=129605
added up to offset=300000, total indexed=155391
added up to offset=350000, total indexed=181244
added up to offset=400000, total indexed=207377
added up to offset=450000, total indexed=233406
added up to offset=500000, total indexed=259220
added up to offset=550000, total indexed=285054
added up to offset=600000, total indexed=311043
added up to offset=650000, total indexed=337165
added up to offset=700000, total indexed=363230
added up to offset=750000, total indexed=389039
added up to offset=800000, total indexed=415029
added up to offset=850000, total indexed=439962


In [7]:

model = CBF_by_embeding()
model.fit(content_based_train, index, all_item_ids)

In [8]:
evaluate_model(model, train_df, test_df)

100%|████████████████████████████████████████████████████████████████████████████████| 4999/4999 [00:44<00:00, 112.58it/s]

Mean Recall@10: 0.0055666610202803396
Mean NDCG@10: 0.01414445604390566


## Ранжирование

Ранжирование (Learning to Rank) — это задача, в которой модель не просто предсказывает значение, а упорядочивает объекты так, чтобы наиболее релевантные оказались выше в списке. Такой подход используется в поиске, рекомендациях и подборе контента — везде, где важна правильная сортировка результатов.

CatBoost хорошо подходит для ранжирования, потому что:

* поддерживает современные алгоритмы LTR (YetiRank, PairLogit и др.);
* отлично работает с категориальными признаками без дополнительной обработки;
* показывает высокую устойчивость и качество «из коробки»;
* легко использовать — достаточно передать группы (group_id).

Поэтому CatBoostRanker — удобная и мощная модель для построения систем рекомендаций и поиска.

### Гибридная модель

Объединием все полученные модели в одну гибридную. Каждая будет выдвать свой топ предсказаний ( но будем брать не 10 а больше). Затем полученные результаты будут уходить на вход модели ранжирования

In [107]:
from catboost import CatBoostRanker

class HybridModel:
    def __init__(self):
        self.trend_model = MostPop_by_likes()
        self.recent_activity_model = RecentAactivityBasedRecommendation()
        self.als_model = ALS()
        self.embeding_model = CBF_by_embeding()

        self.new_items_model = FavoriteArtistsNewReleases()
        self.N = 10

    
    def fit(self, df, df_merged, conf_train_data, content_based_train, index, all_item_ids):
        print("Prepere data")
        last_5days = df[df['timestamp'] > (df['timestamp'].max() - 5 * DAY_SECONDS)]
        last_20days = df_merged[df_merged['timestamp'] > (df_merged['timestamp'].max() - 20 * DAY_SECONDS)]

        user_item_relevance = (
            conf_train_data
            .set_index(["uid", "item_id"])["conf"]
            .to_dict()
        )

        hybrid_train_users = conf_train_data["uid"].unique().tolist()

        matrix = create_user_item_matrix(conf_train_data)

        
        print("Train recomended models")
        print("trend_model")
        self.trend_model.fit(last_5days)

        self.new_items_model.fit(df_merged)
        print("recent_activity_model")
        self.recent_activity_model.fit(last_20days)
        print("als_model")
        self.als_model.fit(matrix)
        print("embeding_model")

        self.embeding_model.fit(content_based_train, index, all_item_ids)

        
        print("Prepere data for rancin model")
        
        X_rows = []      # сюда складываем фичи (строки)
        y_train = []     # сюда релевантность
        group_id = []    # группы (по пользователю)
        
        current_group = 0

        for user_id in tqdm(hybrid_train_users):
            # кандидаты от гибридной системы
            candidates = self.gather_candidates(user_id)
            if not candidates:
                # если модель не смогла ничего отдать – пропускаем пользователя
                continue
        
            rows = self.make_features(user_id, candidates)
            if not rows:
                continue
        
            n_rows = len(rows)
        
            for row in rows:
                item_id = row["item_id"]
        
                # достаём релевантность, если не нашли — 0
                rel = user_item_relevance.get((user_id, item_id), 0.0)
        
                X_rows.append(row)
                y_train.append(rel)
        
            # одна группа = один пользователь, длина группы = число кандидатов для него
            group_id.extend([current_group] * n_rows)
            current_group += 1
        
        self.X_train = pd.DataFrame(X_rows)
        self.y_train = y_train
        self.group_id = group_id
        
        print("Train rancin model")
        
        self.hybrid_model = CatBoostRanker(
            iterations=300,
            learning_rate=0.05,
            depth=6,
            loss_function="YetiRank",
            verbose=50,
        )
        
    def fit_ranker(self):
        self.hybrid_model.fit(
            self.X_train,
            self.y_train,
            group_id=self.group_id,
        )


    def gather_candidates(self, uid):
        rec1 = self.trend_model.recommend(uid, self.N)
        rec2 = self.recent_activity_model.recommend(uid, self.N)
        
        
        rec3 = self.als_model.recommend(uid, self.N)
        rec4 = self.embeding_model.recommend(uid, self.N)
        rec5 = self.new_items_model.recommend(uid, self.N)


        # candidates: {item: {score_model1, score_model2, ...}}
        candidates = {}

        items, scores = rec1  
        for item, score in zip(items, scores):
            candidates.setdefault(item, {})['trend'] = score
    
        items, scores = rec2  
        for item, score in zip(items, scores):
            candidates.setdefault(item, {})['recent_activity'] = score

        items, scores = rec3  
        for item, score in zip(items, scores):
            candidates.setdefault(item, {})['als'] = score

        items, scores = rec4  
        for item, score in zip(items, scores):
            candidates.setdefault(item, {})['embed'] = score

        items, scores = rec5  
        for item, score in zip(items, scores):
            candidates.setdefault(item, {})['new'] = score
            
        return candidates

    def make_features(self, user_id, candidates):
        rows = []
        for item_id, scores in candidates.items():
    
            row = {
                "user_id": user_id,
                "item_id": item_id,
        
                "score_trend":      scores.get("trend", 0),
                "score_recent_activity":   scores.get("recent_activity", 0),
                "score_als":   scores.get("als", 0),
                "score_embed":   scores.get("embed", 0),
                "score_new":   scores.get("new", 0),
            }
    
            rows.append(row)
        
        return rows
            
    def recommend(self, uid, k = 10):
        """
        Инференс мета-ранжировщика:
        - hybrid.gather_candidates(user_id) — отдает кандидатов от базовых моделей
        - make_features(user_id, candidates) — строит те же фичи, что и на обучении
        - model.predict(...) — CatBoostRanker, обученный ранее
        """
    
        # 1. Собираем кандидатов от гибридной системы
        candidates = self.gather_candidates(uid)
        if not candidates:
            # Если для пользователя нет кандидатов — можно вернуть пустой список
            return []
    
        # 2. Строим фичи для мета-модели
        rows = self.make_features(uid, candidates)
        if not rows:
            return []
    
        df = pd.DataFrame(rows)
    
        # Важно: набор колонок должен совпадать с X_train, на котором обучали модель
        # Если нужно, можно явно отсортировать/отфильтровать колонки под training schema:
        # df = df[X_train.columns]
    
        # 3. Предсказываем score для каждого (user, item)
        scores = self.hybrid_model.predict(df)
        df["score"] = scores
    
        # 4. Сортируем по score и возвращаем top_k item_id
        df_sorted = df.sort_values("score", ascending=False)
    
        # Итоговый список item'ов
        top_items = df_sorted["item_id"].head(k).tolist()
    
        return top_items, df_sorted  


In [49]:
import numpy as np
import pandas as pd

train_df_als = merge_data_by_count(train_df)

train_df_als["conf"] = (
    (train_df_als["have_like"] * 20.0)
    + (train_df_als["have_dislike"] * -10.0)
    + np.log1p(
        train_df_als["listen_count"] * (train_df_als["played_ratio_pct"] / 100.0)
    )
)

In [50]:
content_based_train = train_df_als[(train_df_als["have_dislike"]==0)][(train_df_als["listen_count"]>5) | (train_df_als["have_like"]==1)]

/tmp/ipykernel_27294/1560385644.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  content_based_train = train_df_als[(train_df_als["have_dislike"]==0)][(train_df_als["listen_count"]>5) | (train_df_als["have_like"]==1)]


In [52]:
index, all_item_ids = create_index("data/filtered_embeddings.parquet", item_id_map)

added up to offset=100000, total indexed=51853
added up to offset=150000, total indexed=77962
added up to offset=200000, total indexed=103823
added up to offset=250000, total indexed=129605
added up to offset=300000, total indexed=155391
added up to offset=350000, total indexed=181244
added up to offset=400000, total indexed=207377
added up to offset=450000, total indexed=233406
added up to offset=500000, total indexed=259220
added up to offset=550000, total indexed=285054
added up to offset=600000, total indexed=311043
added up to offset=650000, total indexed=337165
added up to offset=700000, total indexed=363230
added up to offset=750000, total indexed=389039
added up to offset=800000, total indexed=415029
added up to offset=850000, total indexed=439962


In [108]:
hybrid = HybridModel()
hybrid.fit(train_df, df_merged, train_df_als, content_based_train, index, all_item_ids)
hybrid.fit_ranker()

Prepere data
Train recomended models
trend_model
recent_activity_model
als_model


100%|█████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:29<00:00,  1.02it/s]


embeding_model
Prepere data for rancin model


100%|█████████████████████████████████████████████████████████████████████████████████| 9431/9431 [10:59<00:00, 14.30it/s]


Train rancin model
0:	total: 282ms	remaining: 1m 24s
50:	total: 8.85s	remaining: 43.2s
100:	total: 17.2s	remaining: 33.9s
150:	total: 25.7s	remaining: 25.3s
200:	total: 33.6s	remaining: 16.5s
250:	total: 41.5s	remaining: 8.09s
299:	total: 48.9s	remaining: 0us


In [40]:
hybrid.hybrid_model = CatBoostRanker(
    iterations=300,
    learning_rate=0.05,
    depth=6,
    loss_function="YetiRank",
    verbose=50,   # выводим метрики каждые 50 итераций
)


hybrid.fit_ranker()

0:	total: 368ms	remaining: 1m 49s
50:	total: 7.29s	remaining: 35.6s
100:	total: 14.2s	remaining: 28s
150:	total: 21.3s	remaining: 21s
200:	total: 28.6s	remaining: 14.1s
250:	total: 36s	remaining: 7.03s
299:	total: 42.8s	remaining: 0us


In [109]:

evaluate_model(hybrid, train_df, test_df)

100%|█████████████████████████████████████████████████████████████████████████████████| 4989/4989 [06:47<00:00, 12.24it/s]

Mean Recall@10: 0.08874351958598513
Mean NDCG@10: 0.1960744376868376


In [26]:
hybrid.X_train['score_embed'].value_counts()

0.000000    221480
0.687744         8
0.484972         8
0.559171         7
0.494140         6
             ...  
0.765191         1
0.757849         1
0.756149         1
0.873573         1
0.789946         1
Name: score_embed, Length: 61887, dtype: int64